In [2]:
import pandas as pd
import numpy as np

In [3]:
import pickle

with open('validation_test.pickle', 'rb') as handle:
    a = pickle.load(handle)
(valid_X,valid_Y,test_X,test_Y) = a

with open('scaler.pickle', 'rb') as handle:
    scaler = pickle.load(handle)


In [4]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
import tensorflow.keras.layers as layers

In [124]:
checkpoint_filepath = './checkpoints/'
#load checkpoint model
model_filename = ['run-0_20200603-180616_model.hdf5','run-0_20200603-174059_model.hdf5', 'run-0_20200603-170545_model.hdf5','run-0_20200531-151655_model.hdf5',"run-0_20200531-144203_model.hdf5"]
model = tf.keras.models.load_model(checkpoint_filepath + model_filename[0])
model.summary()

Model: "sequential_50"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_4 (Bidirection multiple                  3312      
_________________________________________________________________
batch_normalization_118 (Bat multiple                  144       
_________________________________________________________________
bidirectional_5 (Bidirection multiple                  7920      
_________________________________________________________________
batch_normalization_119 (Bat multiple                  144       
_________________________________________________________________
dense_99 (Dense)             multiple                  1184      
_________________________________________________________________
batch_normalization_120 (Bat multiple                  128       
_________________________________________________________________
dropout_71 (Dropout)         multiple                

In [116]:
valid_loss, valid_acc = model.evaluate(valid_X, valid_Y, verbose=2) # 5/2020 nt: use validation set
print ("valid_accuracy_MAE=%s, valid_loss=%s" % (valid_acc, valid_loss))

9547/9547 - 8s - loss: 0.0014 - mae: 0.0272
valid_accuracy_MAE=0.027222563, valid_loss=0.0014464554635752658


In [8]:
def padding(data, nopad):
    flat = data.flatten()
    padding = np.zeros((flat.shape[0],nopad))
    for i,row in enumerate(padding):
        padding[i,1] = flat[i]
    print(padding.shape)
    data_out = scaler.inverse_transform(padding)
    return np.reshape(data_out, (int(data_out.shape[0]/data.shape[1]),data.shape[1],nopad))

In [118]:
pred = model.predict(test_X)
pred.shape
padded = padding(pred, 4)
df = pd.read_csv('climate_hour.csv')
get_index = df.iloc[-21984:]['Date Time'].values
str_index = [index.replace(' ','_') for index in get_index]
result_df = pd.DataFrame(padded[:,0,1], columns=['temperature'], index=str_index)
result_df.index = result_df.index.rename('date_time')
diff = result_df['temperature'].values - df.iloc[-21984:]['T (degC)'].values
np.average(np.abs(diff))

(21984, 1)

In [105]:
result_df.to_csv('results.csv')

#### new LSTM model

In [209]:
checkpoint_filepath = './checkpoints/'
#load checkpoint model
model_filename = ['_20200604-160756_model.hdf5','_20200604-160310_model.hdf5','_20200604-154549_model.hdf5','_20200604-152527_model.hdf5','_20200604-145243_model.hdf5','_20200604-134707_model.hdf5','best_model.hdf5','run-0_20200603-180616_model.hdf5','run-0_20200603-174059_model.hdf5', 'run-0_20200603-170545_model.hdf5','run-0_20200531-151655_model.hdf5',"run-0_20200531-144203_model.hdf5"]
model2 = tf.keras.models.load_model(checkpoint_filepath + model_filename[0])
model2.summary()

Model: "sequential_133"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_358 (LSTM)              (None, 48, 10)            600       
_________________________________________________________________
lstm_359 (LSTM)              (None, 48, 6)             408       
_________________________________________________________________
lstm_360 (LSTM)              (None, 1)                 32        
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 1)                 0         
_________________________________________________________________
repeat_vector_39 (RepeatVect (None, 48, 1)             0         
_________________________________________________________________
lstm_361 (LSTM)              (None, 48, 48)            9600      
_________________________________________________________________
time_distributed_38 (TimeDis (None, 48, 1)          

In [210]:
valid_loss, valid_acc = model2.evaluate(valid_X, valid_Y, verbose=2) # 5/2020 nt: use validation set
print ("valid_accuracy_MAE=%s, valid_loss=%s" % (valid_acc, valid_loss))

9547/9547 - 7s - loss: 0.0012 - mae: 0.0259
valid_accuracy_MAE=0.025920218, valid_loss=0.0011677236298833818


### 1.56

In [211]:
pred = model2.predict(test_X)
pred.shape
padded = padding(pred, 4)
df = pd.read_csv('climate_hour.csv')
get_index = df.iloc[-21984:]['Date Time'].values
str_index = [index.replace(' ','_') for index in get_index]
result_df = pd.DataFrame(padded[:,0,1], columns=['temperature'], index=str_index)
result_df.index = result_df.index.rename('date_time')
diff = result_df['temperature'].values - df.iloc[-21984:]['T (degC)'].values
np.average(np.abs(diff))

(21984, 4)


1.5816080175659024

#### 1.5515798884664904

In [205]:
#result_df.to_csv('results2.csv')

In [32]:
checkpoint_filepath = './checkpoints/'
#load checkpoint model
model_filename = ['_20200605-153441_model.hdf5','_20200604-170111_model.hdf5','_20200604-160756_model.hdf5','_20200604-160310_model.hdf5','_20200604-154549_model.hdf5','_20200604-152527_model.hdf5','_20200604-145243_model.hdf5','_20200604-134707_model.hdf5','best_model.hdf5','run-0_20200603-180616_model.hdf5','run-0_20200603-174059_model.hdf5', 'run-0_20200603-170545_model.hdf5','run-0_20200531-151655_model.hdf5',"run-0_20200531-144203_model.hdf5"]
model3 = tf.keras.models.load_model(checkpoint_filepath + model_filename[0])
model3.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_66 (LSTM)               (None, 48, 20)            2000      
_________________________________________________________________
lstm_67 (LSTM)               (None, 48, 12)            1584      
_________________________________________________________________
lstm_68 (LSTM)               (None, 2)                 120       
_________________________________________________________________
repeat_vector_15 (RepeatVect (None, 48, 2)             0         
_________________________________________________________________
lstm_69 (LSTM)               (None, 48, 48)            9792      
_________________________________________________________________
time_distributed_15 (TimeDis (None, 48, 1)             49        
_________________________________________________________________
dropout (Dropout)            (None, 48, 1)           

In [33]:
valid_loss, valid_acc = model3.evaluate(valid_X, valid_Y, verbose=2) # 5/2020 nt: use validation set
print ("valid_accuracy_MAE=%s, valid_loss=%s" % (valid_acc, valid_loss))

9547/9547 - 7s - loss: 0.0257 - mae: 0.0257
valid_accuracy_MAE=0.025678743, valid_loss=0.025678738899785735


In [34]:
pred = model3.predict(test_X)
pred.shape
padded = padding(pred, 4)
df = pd.read_csv('climate_hour.csv')
get_index = df.iloc[-21984:]['Date Time'].values
str_index = [index.replace(' ','_') for index in get_index]
result_df = pd.DataFrame(padded[:,0,1], columns=['temperature'], index=str_index)
result_df.index = result_df.index.rename('date_time')
diff = result_df['temperature'].values - df.iloc[-21984:]['T (degC)'].values
np.average(np.abs(diff))

(21984, 4)


1.538613192959644

In [35]:
result_df.to_csv('results.csv')